In [2]:
# creating a Sales Developmenr Representative (SDR) ai agent using LangChain and Groq
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.tools import tool
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

### print the environment variables
print("Environment Variables:")
for key, value in os.environ.items():
    print(f"{key}: {value}")
    

Environment Variables:
ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\Kartik\AppData\Roaming
APPLICATION_INSIGHTS_NO_STATSBEAT: true
CHROME_CRASHPAD_PIPE_NAME: \\.\pipe\crashpad_32480_SDBSQEEFWFWVCUDY
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: DESKTOP-HGGGQ7A
COMSPEC: C:\WINDOWS\system32\cmd.exe
CONDA_ALLOW_SOFTLINKS: false
CONDA_DEFAULT_ENV: C:\Users\Kartik\OneDrive\Desktop\aiml\venv
CONDA_EXE: C:\Users\Kartik\anaconda3\Scripts\conda.exe
CONDA_PREFIX: C:\Users\Kartik\OneDrive\Desktop\aiml\venv
CONDA_PROMPT_MODIFIER: (C:\Users\Kartik\OneDrive\Desktop\aiml\venv) 
CONDA_PYTHON_EXE: C:\Users\Kartik\anaconda3\python.exe
CONDA_ROOT: C:\Users\Kartik\anaconda3
CONDA_SHLVL: 1
CUDA_PATH: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5
CUDA_PATH_V12_5: C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v12.5
DEBUG: WARN
DRIVERDATA: C:\Windows\System32\Dr

In [21]:
os.environ["HUGGINGFACE_TOKEN"] = os.getenv("HUGGINGFACE_TOKEN")
os.environ["LANGSMTIH_TOKEN"] = os.getenv("LANGSMTIH_TOKEN")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langsmith.ai"
os.environ["LANGCHAIN_PROJECT"] = "pr-new-hobby-13"
os.environ["GROQ_API_KEY"] = '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'



In [26]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})



In [27]:
# documents
texts

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith. Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architect

In [28]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="langchain_docs",
    description="Useful for answering questions about LangSmith and LangChain.",
)

retriever_tool ### this is the tool we will use in the agent to answer questions about LangSmith and LangChain

Tool(name='langchain_docs', description='Useful for answering questions about LangSmith and LangChain.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000025145ADC5E0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000251E0061EB0>, search_kwargs={'k': 2}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x0000025145ADC7C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000251E0061EB0>, search_kwargs={'k': 2}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={

In [29]:
retriever_tool.run("What is LangSmith?") ### this is how we can use the tool to answer questions about LangSmith and LangChain

"Get started with LangSmith | 🦜️🛠️ LangSmith\n\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.\nLangSmith + LangChain OSSLangSmith is framework-agnostic —\xa0it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.\nFor more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.\nObservability\u200b"

In [30]:
### lead management tool
import json
from datetime import datetime

LEADS_FILE = "leads.json"

@tool
def save_lead_info(info: str) -> str:
    """
    Save structured lead info in the format:
    "Company: ABC Corp, Domain: Healthcare, Problem: Want better patient tracking, Budget: $10,000"
    """
    try:
        parsed = {}
        for line in info.split(","):
            key, val = line.strip().split(":", 1)
            parsed[key.strip().lower()] = val.strip()
        parsed["timestamp"] = str(datetime.now())

        with open(LEADS_FILE, "a") as f:
            f.write(json.dumps(parsed) + "\n")

        return "Lead information saved successfully!"
    except Exception as e:
        return f"Error saving lead info: {str(e)}"

In [31]:
### media trigger tool for LangChain and LangSmith demos
@tool
def show_demo_media(context: str) -> str:
    """
    Given a product use-case or industry context, return a video or image demo link.
    Example input: "Example Demo", "Langsmith Dashboard", "LangSmith for AI Agents Video"
    """
    media_map = {
        "langsmith dashboard": "https://youtu.be/LEIghyTGgQk?feature=shared",
        "langsmith for ai agents video": "https://youtu.be/gOK65vR0hIY?feature=shared",
        "Example demo": "https://youtu.be/kYtnLaJeia8?feature=shared",
    }

    for keyword, url in media_map.items():
        if keyword in context.lower():
            return f"Here’s a quick demo for {keyword}: {url}"

    return "Sorry, I couldn’t find a matching demo right now."

In [32]:
tools = [retriever_tool, save_lead_info, show_demo_media] ### set of tools we will use in the agent

In [33]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""
Greet the user. Then ask for the following, one at a time: 
1. Company name 
2. Domain 
3. Problem statement 
4. Budget

Once you have all four, summarize and call the save_lead_info tool.

If they ask a product-related question, call the retriever_tool.
If they ask for a demo, call the show_demo_media tool.
If they ask anything other than these, respond witha friendly message acknowledging their question and offering to help with product info or demos.
""")

In [34]:
### setting up a memory
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model

### selecting the chat model
model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [35]:
### binding tools to the agent chat model
model_with_tools = model.bind_tools(tools)

In [36]:
### agent executor

memory = MemorySaver()
agent_executor = create_react_agent(
    model=model_with_tools,
    prompt=prompt_template,
    tools=tools,
    checkpointer=memory
)

config = {"configurable": {"thread_id": "sdr_agent"}}

In [37]:
### try agent
from langchain_core.messages import HumanMessage
response= agent_executor.invoke(
   { "messages":[
        HumanMessage(
            content="What is LangSmith? Can you also show me a demo of your product?"
        )
    ]}, config
)
print(response["messages"]) ### this is the response from the agent

[HumanMessage(content='What is LangSmith? Can you also show me a demo of your product?', additional_kwargs={}, response_metadata={}, id='2426e549-c0fd-43a3-8416-07ed8c390991'), AIMessage(content="Hello! I'm excited to help you. \n\nCan you please tell me what company you are from?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 2512, 'total_tokens': 2534, 'completion_time': 0.018333333, 'prompt_time': 0.313216752, 'queue_time': -0.373915563, 'total_time': 0.331550085}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run--9db5fff3-f924-4394-ad98-042e9db8faf2-0', usage_metadata={'input_tokens': 2512, 'output_tokens': 22, 'total_tokens': 2534})]


In [38]:
for chunk in agent_executor.stream (
   { "messages":[
        HumanMessage(
            content="hello I need help with my lead management. Can you help me save a lead?"
        )
    ]}, config
):
    print(chunk)
    print("----")
# print(response["messages"])

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xh3k', 'function': {'arguments': '{"query":"Hello, I\'d like to learn more about LangSmith and LangChain."}', 'name': 'langchain_docs'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 1246, 'total_tokens': 1352, 'completion_time': 0.088333333, 'prompt_time': 0.180333805, 'queue_time': 0.033526362999999976, 'total_time': 0.268667138}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b4e75c7a-6da9-4b1e-b137-7e028e4322bd-0', tool_calls=[{'name': 'langchain_docs', 'args': {'query': "Hello, I'd like to learn more about LangSmith and LangChain."}, 'id': 'call_xh3k', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1246, 'output_tokens': 106, 'total_tokens': 1352})]}}
----
{'tools': {'messages': [ToolMessage(content="Get started with LangSmith | 🦜️🛠️ LangSmith\n\

In [39]:
from langchain.memory import ConversationBufferMemory
from langchain.agents.agent import AgentExecutor
from langchain.agents import initialize_agent, AgentType

llm = ChatGroq(model="llama-3.1-8b-instant", api_key=os.getenv("GROQ_API_KEY"))


In [40]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\Kartik\AppData\Local\Temp\ipykernel_16640\3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [41]:
agent_executor2 = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

C:\Users\Kartik\AppData\Local\Temp\ipykernel_16640\3504736060.py:1: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor2 = initialize_agent(


In [42]:
print(agent_executor2.run("hello I need help with my lead management. Can you help me save a lead?"))

C:\Users\Kartik\AppData\Local\Temp\ipykernel_16640\2672174587.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(agent_executor2.run("hello I need help with my lead management. Can you help me save a lead?"))




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: save_lead_info
Action Input: "Company: XYZ Inc, Domain: Technology, Problem: Want to automate sales process, Budget: $5,000"
Observation: Error saving lead info: not enough values to unpack (expected 2, got 1)
Thought:Thought: Do I need to use a tool? Yes
Action: save_lead_info
Action Input: "Company: XYZ Inc, Domain: Technology, Problem: Want to automate sales process, Budget: $5,000"
Observation: Error saving lead info: not enough values to unpack (expected 2, got 1)
Thought:Thought: Do I need to use a tool? No
AI: It looks like there's a problem saving the lead info. Can you provide more details about the lead, such as the company name, domain, problem they're trying to solve, and budget? This will help me assist you better. What's the issue you're experiencing with automating the sales process?

> Finished chain.
It looks like there's a problem saving the lead info. Can you provide more details ab

In [46]:
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate

system_template = """
You are Ava, a friendly SDR AI voice assistant.
Your job is to greet the user, ask for their company name, domain, problem, and budget.
Answer product questions using the retriever_tool.
If they request a demo, use the show_demo_media tool.
If you use the show_demo_media tool, provide a media (vieo/image) link within the text so that user can access the link.
Once all information is gathered, use save_lead_info and end the conversation politely.
"""

# Use the llm variable already defined, and pass your system_template as part of the prompt if needed.
# The initialize_agent function does not take a system_message argument directly.
# If you want to customize the system prompt, use the prompt argument in initialize_agent or PromptTemplate.

agent_executor2 = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
    prompt=prompt_template,  # Use the prompt_template defined earlier
        # You can add a custom prompt here if needed, e.g.:
    # prompt=PromptTemplate(input_variables=[], template=system_template)
)

In [44]:
# print(agent_executor2.run("hello I need help with my lead management. Can you help me save a lead?"))
print(agent_executor2.run("What is LangSmith? Can you also show me a demo of your product?"))



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: langchain_docs
Action Input: "LangSmith"
Observation: Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.
LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with Lang

In [19]:
print(agent_executor2.run("I own a healthcare company called ABC Corp. We want to improve patient tracking. Our budget is $10,000."))



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: save_lead_info
Action Input: "Company: ABC Corp, Domain: Healthcare, Problem: Want better patient tracking, Budget: $10,000"
Observation: Error saving lead info: not enough values to unpack (expected 2, got 1)
Thought:Thought: Do I need to use a tool? Yes
Action: save_lead_info
Action Input: "Company: ABC Corp, Domain: Healthcare, Problem: Want better patient tracking, Budget: $10,000"
Action: langchain_docs
Action Input: lead info
Observation: Lead information saved successfully!
Thought:Thought: Do I need to use a tool? No
AI: Based on the lead information you provided, it seems like ABC Corp is looking to improve patient tracking in the healthcare industry. I'd be happy to help you explore ways to achieve this goal.

Given your budget of $10,000, we can consider various solutions that might be suitable for your company. One option could be using LangSmith's Observability feature, which can help you

In [47]:
# print(agent_executor2.run("What is my lead information you stored?"))
print(agent_executor2.run("Can you show me a demo of your product?"))



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: show_demo_media
Action Input: "Langsmith Dashboard"
Observation: Here’s a quick demo for langsmith dashboard: https://youtu.be/LEIghyTGgQk?feature=shared
Thought:Thought: Do I need to use a tool? No
AI: I've shown you a demo of the Langsmith Dashboard. As I mentioned earlier, LangSmith is a platform for building production-grade LLM applications. It offers features such as observability, evals, and prompt engineering. Let me know if you have any further questions or if there's anything else I can help you with. If you'd like to see more demos or explore specific features of LangSmith, feel free to ask!

> Finished chain.
I've shown you a demo of the Langsmith Dashboard. As I mentioned earlier, LangSmith is a platform for building production-grade LLM applications. It offers features such as observability, evals, and prompt engineering. Let me know if you have any further questions or if there's anythi